In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.

In [ ]:
import pandas as pd
df_product=pd.read_csv("/content/drive/MyDrive/LLMDatasets/dataset1/products.csv")
df_dept=pd.read_csv("/content/drive/MyDrive/LLMDatasets/dataset1/departments.csv")

In [ ]:
df_joined=pd.merge(df_product, df_dept, on = ['department_id'])
df_joined['text']=df_joined.apply(lambda row: row['product_name'] + " ->: " + row['department'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df=train_test_split(df_joined, test_size=0.2, random_state=42)

In [ ]:
from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df),
})

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name ="TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model=AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache=False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
import transformers
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


sequences=pipeline(
   ["“Free & Clear Stage 4 Overnight Diapers” ->:","Bread Rolls ->:","French Milled Oval Almond Gourmande Soap ->:"],
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Result: “Free & Clear Stage 4 Overnight Diapers” ->: http://amzn. październik 2017 – 2 min – Uploaded by BabyGearLabBabyGearLab.com: We test, compare and analyze all the top diapers in the United States in order to ….
Free & Clear Stage 4 Overnight Diapers
Free & Clear Stage 4 Overnight Diapers 16.49/10.5 oz. 2016-07-11 · Huggies Overnight Protective Night Pants, Size 6-18 Months, Baby Pink/Gold Huggies, 23 Count (Packaging May …. 2017-01-05 · Free & Clear Stage 4 diapers are a good choice for sensitive skin and are the best overnight diapers on the market. 0
Result: Bread Rolls ->: 5. nobody.
13. What is a good thing that you can find in the supermarket ->: 5. bread.
14. How long has it been ->: 5. I am not sure.
15. How often do you go to McDonald’s ->: 5. every 2 weeks.
16. What is the best place you have been on holiday? ->: 5. I like to stay at home.
18. Do you like chess? ->: 5. no I do not.
20. Is it easy for you to make friends? ->: 5. no.
21. Do you like reading books? ->: 5. 

In [ ]:
from peft import LoraConfig

lora_alpha=16
lora_dropout=0.1
lora_r=64

peft_config=LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj"]
)

In [ ]:
from transformers import TrainingArguments

output_dir="./results"
per_device_train_batch_size=4
gradient_accumulation_steps=4
optim="paged_adamw_32bit"
save_steps=10
logging_steps=1
learning_rate=2e-4
max_grad_norm=0.3
max_steps=120
warmup_ratio=0.03
lr_scheduler_type="constant"

training_arguments=TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
from trl import SFTTrainer

max_seq_length=512

trainer=SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:104: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/39750 [00:00<?, ? examples/s]

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module=module.to(torch.float32)

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.029100
2,3.089600
3,4.359200
4,6.271200
5,4.250100
6,3.093500
7,4.517900
8,3.970400
9,4.471600
10,5.359200


TrainOutput(global_step=120, training_loss=2.6914264718691507, metrics={'train_runtime': 577.8182, 'train_samples_per_second': 3.323, 'train_steps_per_second': 0.208, 'total_flos': 627303342243840.0, 'train_loss': 2.6914264718691507, 'epoch': 0.05})

In [ ]:
lst_test_data=list(test_df['text'])

In [ ]:
sample_size=25
lst_test_data_short=lst_test_data[:sample_size]

In [ ]:
import transformers

pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    lst_test_data_short,
    max_length=100,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


0 Free & Clear Stage 4 Overnight Diapers ->: babies toddlers kids baby toiletries free clear stage 4 overnight diapers ->: babies toddlers kids kids toys free clear stage 4 overnight diapers ->: toys games toddlers toys ->: personal hygiene baby care ->: baby care personal hygiene babies toddlers kids free clear stage 4 overnight dia
1 Beef pot roast with roasted potatoes, carrots, sweet onions, green beans, and a rich gravy Beef Pot Roast ->: frozen foods pasta and meat ->: meat products beef, pork meat products beef, pork ->: meats, poultry, seafood, cheese meat products beef, pork ->: meats, poultry, seafood, cheese: meat products be
2 Coffee Liquer ->: alcohol spirits coffee liqueur ->: beverages non-alcoholic beverages coffee beverages ->: coffee, espresso & espresso drinks coffee liqueuers: coffee liqueuer ->: beverages non-alcoholic beverages coffee beverages: coconut coffee ->: beverages non-alcoholic beverages coffee beverages ->: bever
3 Bread Rolls ->: bakery ->: snacks ->: 

In [ ]:
def correct_answer(ans):
  return (ans.split("->:")[1]).strip()

answers=[]
for ix,seq in enumerate(sequences):
    answers.append(correct_answer(seq[0]['generated_text']))

answers

['babies toddlers kids baby toiletries free clear stage 4 overnight diapers',
 'frozen foods pasta and meat',
 'alcohol spirits coffee liqueur',
 'bakery',
 'personal care beauty soaps soap bar',
 'household: cleaning',
 'deli meat & cheese',
 'beverages: soft drinks & waters',
 'dairy eggs & dairy ice cream dairy products butter',
 'canned goods soup',
 'dairy eggs eggs eggs',
 'snacks candy bars',
 'dry goods pasta',
 'personal care babies children',
 'beverages',
 'international pantry dry goods spices, seasoning, sauces & soups',
 'breakfast',
 'dairy eggs eggs',
 'meat seafood',
 'household cleaning personal care toiletries votive candles',
 'frozen foods',
 'dairy eggs & cheese eggs & dairy eggs & beans',
 'breakfast foods',
 'personal care toiletries & personal care products facial care',
 'meat seafood pasta']

In [ ]:
df_evaluate=test_df.iloc[:sample_size][['product_name','department']]

df_evaluate=df_evaluate.reset_index(drop=True)

df_evaluate['department_predicted']=answers
df_evaluate

,product_name,department,department_predicted
0,Free & Clear Stage 4 Overnight Diapers,babies,babies toddlers kids baby toiletries free clea...
1,"Beef pot roast with roasted potatoes, carrots,...",frozen,frozen foods pasta and meat
2,Coffee Liquer,alcohol,alcohol spirits coffee liqueur
3,Bread Rolls,bakery,bakery
4,French Milled Oval Almond Gourmande Soap,personal care,personal care beauty soaps soap bar
5,Dust Pan,household,household: cleaning
6,Roasted Pine Nut Hommus,deli,deli meat & cheese
7,Cranberry Raspberry Juice Cocktail,beverages,beverages: soft drinks & waters
8,Sweet Cream Butter Salted,dairy eggs,dairy eggs & dairy ice cream dairy products bu...
9,Traditional Chicken Barley Soup,canned goods,canned goods soup
